In [29]:
from os import listdir
from os.path import isfile
from json import load
from shutil import rmtree
from shutil import copytree
from os import makedirs

In [30]:
PASTA = "requisicoes"    
ORDEM_DA_LUZ = 'ordem_da_luz'

In [31]:
total_respondidas = 0
total_sem_responder  = 0
for item in listdir(PASTA):
    caminho = f'{PASTA}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
    if isfile(caminho_resposta):
        total_respondidas+=1
    if not isfile(caminho_resposta):
        print("sem responder em: ", item)
        total_sem_responder+=1


print("total: ", total_respondidas + total_sem_responder)
print("total respondidas: ",total_respondidas)
print("total sem responder",total_sem_responder)


total:  13
total respondidas:  13
total sem responder 0


In [32]:
clientes = {
    "1":{
        "saldo":0,
        "limite":100000,
    },
    "2":{
        "saldo":0,
        "limite":80000
    },
    "3":{
        "saldo":0,
        "limite":1000000
    },
    "4":{
        "saldo":0,
        "limite":10000000
    },
    "5":{
        "saldo":0,
        "limite":500000
    }
}
ids = clientes.keys()

Organizando a ordem


In [33]:
rmtree(ORDEM_DA_LUZ,ignore_errors=True)
makedirs(ORDEM_DA_LUZ)

listagem = listdir(PASTA)
for item in listagem:
    caminho = f'{PASTA}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
    if not isfile(caminho_resposta):
        continue

    with open(caminho_resposta,'r') as arq:
        respostas = load(arq)


    if not respostas.get('json_body'):
        continue
    start = f'{respostas["luz_adiquirida"]["segundos"]}:{respostas["luz_adiquirida"]["nanosegundos"]}'

    end = ''
    if respostas.get('luz_liberada'):
        end = f'{respostas["luz_liberada"]["segundos"]}:{respostas["luz_liberada"]["nanosegundos"]}'
        
    novo_caminho =  f'{ORDEM_DA_LUZ}/{start}=>{end}|{item}'
    copytree(caminho,novo_caminho)

        



In [34]:
listagem = listdir(ORDEM_DA_LUZ)
listagem.sort()

for item in listagem:

    caminho = f'{ORDEM_DA_LUZ}/{item}'
    caminho_request = f'{caminho}/request.json'
    with open(caminho_request,'r') as arq:
        request = load(arq)
    
    caminho_resposta = f'{caminho}/resposta.json'
    if not isfile(caminho_resposta):
        raise Exception('aaaaaaa')
    
    rota:str = request['rota']

    rota_dividida = rota.split('/')
    id_cliente = rota_dividida[2]
    if id_cliente not in ids:
        raise Exception("aaaaaaaa")
    
    cliente_atual = clientes[id_cliente]
    
    tipo_rota = rota_dividida[3]
    if tipo_rota != 'transacoes':
        continue

    json_body = request['json_body']

    if json_body['valor'] < 0:
        raise  Exception("não deveria passar inteiros negativos")

    if json_body['valor'].__class__ != int:
        raise  Exception("não deveria passar qualquer tipo de valor nã inteiro")

    if  not json_body.get('descricao'):
        raise Exception("nãp ha descricao")


    if len(json_body['descricao']) == 0 or len(json_body['descricao'])> 10:
        raise Exception("descrição com valores inválidos")


    if json_body['tipo'] == 'c':
        cliente_atual['saldo'] += json_body['valor']

    
    if json_body['tipo'] == 'd':
        saldo_e_limite = cliente_atual['saldo'] + cliente_atual['limite']
        if saldo_e_limite - json_body['valor'] < 0:
            continue
        cliente_atual['saldo'] -= json_body['valor']


    caminho_resposta = f'{caminho}/resposta.json'
    with open(caminho_resposta,'r') as arq:
        resposta = load(arq)

    if resposta['json_body']['saldo'] != cliente_atual['saldo']:
        raise Exception('quebrou em ',item)
    

Exception: aaaaaaaa

In [ ]:
clientes

{'1': {'saldo': 0, 'limite': 100000},
 '2': {'saldo': 0, 'limite': 80000},
 '3': {'saldo': 0, 'limite': 1000000},
 '4': {'saldo': 0, 'limite': 10000000},
 '5': {'saldo': 0, 'limite': 500000}}